In [ ]:
%matplotlib inline

#change functions versus objects to camelback
#implement clustering tree given distance matrix
#think about how to choose threshold (should be between distance "modes")
#incorporate insert size distribution

import numpy as np
import pandas as pd
import networkx as nx
from scipy.misc import comb
import scipy.stats as ss
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.mplot3d import Axes3D
from collections import Counter
from __future__ import division

def load_table(sample):
    f = open(sample + '.table.txt', 'r')
    read_groups = eval(f.readline())
    libraries = eval(f.readline())
    n_reads = eval(f.readline())
    n_reads_removed = eval(f.readline())
    table = eval(f.readline())
    f.close()
    return read_groups, libraries, n_reads, n_reads_removed, table

def group_sizes(table, n_groups):
    '''Returns vector v where v[i] is count of reads in group i.'''
    rg_sizes = np.zeros(n_groups, dtype=int)
    for item in table.items():
        for i in item[0]:
            rg_sizes[i] += item[1]
    return rg_sizes

def remove_empty_groups(read_groups, libraries, table):
    n_groups = len(read_groups)
    rg_sizes = group_sizes(table, n_groups)
    if np.prod(rg_sizes != 0):
        return read_groups, libraries, table
    else:
        groups = [read_groups[i] for i in range(n_groups) if rg_sizes[i] != 0]
        libs = [libraries[i] for i in range(n_groups) if rg_sizes[i] != 0]
        new_num_of_old_num = {}
        current_num = 0
        for i in range(n_groups):
            new_num_of_old_num[i] = current_num
            if rg_sizes[i] != 0:
                current_num += 1
        new_table = Counter()
        for item in table.items():
            new_key = tuple(map(lambda x : new_num_of_old_num[x], list(item[0])))
            new_table.update({new_key : item[1]})
        return groups, libs, new_table
    
def library_nums(libraries):
    n_groups = len(libraries)
    num_of_library = {}
    current_num = 0
    for i in xrange(n_groups):
        if libraries[i] not in num_of_library.keys(): #change to set_default
            num_of_library[libraries[i]] = current_num
            current_num += 1
    return np.array([num_of_library[libraries[i]] for i in xrange(n_groups)])

def library_colors(lib_nums):
    #lib_color = plt.cm.Set1(lib_nums / (n_libraries + 1))
    color_of_num = {0 : 'r', 1 : 'b', 2 : 'g', 3 : 'k', 4 : 'y', 5 : 'm', 6 : 'c'}
    return [color_of_num[lib_num % 5] for lib_num in lib_nums]

def library_matrix(lib_nums, n_groups):
    lib_matrix = np.zeros((n_groups,n_groups), dtype=bool)
    for i in xrange(n_groups):
        for j in xrange(n_groups):
            lib_matrix[i,j] = (lib_nums[i] == lib_nums[j])
    return lib_matrix

# jaccard similarity. union/intersection. might not be robust to diff conditions. no a priori expectation
def overlaps(table, n_groups):
    '''Returns matrix where M[i][j] is count of read types in both groups i
       and group j (symmetrized)'''
    overlap_matrix = np.zeros((n_groups, n_groups))
    for item in table.items():
        groups = list(set(item[0]))
        l = len(groups)
        for i in xrange(0,l):
            for j in xrange(i,l):
                overlap_matrix[groups[i],groups[j]] += item[1]
    return (overlap_matrix + overlap_matrix.T) / 2

def inner_product(table, n_groups):
    '''Returns matrix X^t * X where X[i,j] is count of reads of type i in group j.
       Agrees with all_pairs off the diagonal.'''
    ip_matrix = np.zeros((n_groups,n_groups))
    for item in table.items():
        l = len(item[0])
        count_vector = np.zeros(n_groups)
        for i in xrange(l):
            count_vector[item[0][i]] += 1
        ip_matrix += np.outer(count_vector, count_vector) * item[1]
    return ip_matrix


# ABB would be 3 pairs- AB, AB, BB
# Retain signal from non-exact pairs. didn't do this as the only metric bc seeing a lot of on-flowcell pad-hopping duplicates, showing up as large duplicate sets, which would dominate here
# We only want biological duplicates, and we have been better at marking these. The work yossi and david benj are doing. What's going on from david roazen.
# That wasn't ready yet, so he did an exact pairs approach as well.
# This one is probably best if you are careful to look at only biological duplicates
def all_pairs(table, n_groups):
    '''Returns matrix where M[i,j] is count of matching pairs of distinct reads
       from groups i and j (symmetrized).'''
    ap_matrix = np.zeros((n_groups,n_groups))
    for item in table.items():
        l = len(item[0])
        for i in xrange(l-1):
            for j in xrange(i+1,l):
                ap_matrix[item[0][i],item[0][j]] += item[1]
    return (ap_matrix + np.tril(ap_matrix.T))/2

def exact_tuples(table, n_groups, k):
    '''Returns array where A[i_1,...i_k] is count of read types that occur
       exactly in the groups i_1,...,i_k ascending with repeats.'''
    et_array = np.zeros(k*(n_groups,))
    for item in table.items():
        if len(item[0])==k:
            et_array[item[0]] = item[1]
    return et_array

def exact_pairs(table, n_groups):
    '''Returns matrix where M[i,j] is count of read types that occur exactly twice,
       once in group i and once in group j (symmetrized).'''
    ep_matrix = exact_tuples(table, n_groups, 2)
    return (ep_matrix + np.tril(ep_matrix.T))/2

def normalize(array):
    return array/np.sum(array)

# normalize such that diagonals are 1 (so 1 is max self-similarity). off-diag things scaled proportionally
def normalize_by_diagonal(matrix):
    diag = np.diagonal(matrix)
    return matrix / np.sqrt(np.outer(diag, diag))

# take the outer product of [pq] with itself
# ie [[p^2, pq], [pq, q^2]]
# so divide matrix by [[p, rad(pq)], [rad(pq), q]]
# so each cell is divided by the total number of elements in it's thing.
# most similar to the underlying model
def normalize_by_group_size(matrix, group_sizes):
    return matrix / np.sqrt(np.outer(group_sizes, group_sizes))

def normalize_rows_by_vector(matrix, vector):
    return matrix / vector[:, np.newaxis]

def overlap_affinity(table, n_groups):
    ol_matrix = overlaps(table, n_groups)
    ol_affinity = np.zeros((n_groups, n_groups))
    for i in xrange(n_groups):
        ol_affinity[i,i] = .5
        for j in xrange(i+1,n_groups):
            ol_affinity[i,j] = 2 * ol_matrix[i,j] / \
                               (ol_matrix[i,i] + ol_matrix[j,j] - 2 * ol_matrix[i,j])
    return ol_affinity + ol_affinity.T

def exact_pairs_affinity(table, n_groups):
    return normalize_by_diagonal(exact_pairs(table, n_groups))

def all_pairs_affinity(table, n_groups):
    return normalize_by_diagonal(all_pairs(table, n_groups))

# Distance in euclidean space between 2 vectrs. each RG is a vector in a space whose dimension is the number of non-zero entries in the union of observed inserts
# so, what's teh cosine fo the angle between these 2 vectors
# naturally, the diagonal elements will be 1
# cosine of angle = (A dot B) / (length A)(length B)

# BUT our data isn't in the form of these vectors. So we count the AB pairs.
# Recording each time he sees some combination of 2 numbers.

# suffers the same issue as jaccard- what is your expectation. this is why all pairs and exact pairs only viable.
def L2_affinity(table, n_groups):
    return normalize_by_diagonal(inner_product(table, n_groups))

def exact_pairs_affinity2(table, n_groups, rg_sizes):
    return normalize(normalize_by_group_size(exact_pairs(table, n_groups),rg_sizes))

def all_pairs_affinity2(table, n_groups, rg_sizes):
    return normalize(normalize_by_group_size(all_pairs(table, n_groups),rg_sizes))

def zero_diag(matrix):
    new_matrix = matrix.copy()
    for i in xrange(matrix.shape[0]):
        new_matrix[i,i] = 0
    return new_matrix

def multiplicity_matrix(table, n_groups):
    '''Returns M with M_ij count of j-tuples in group i'''
    max_mult =  max(map(len,table.keys()))
    mult_matrix = np.zeros((n_groups, max_mult + 1), dtype=int)
    for i in xrange(n_groups):
        mult_vector = multiplicity_vector(subtable(table,i))
        for j in xrange(len(mult_vector)):
            mult_matrix[i,j] = mult_vector[j]
    return mult_matrix

def multiplicity_vector(table):
    '''Returns v with v_j count of j-tuples in table'''
    max_mult =  max(map(len,table.keys()))
    mult_vector = np.zeros(max_mult + 1, dtype=int)
    for item in table.items():
        mult_vector[len(item[0])] += item[1]   
    return mult_vector

def subtable(table, i):
    '''Returns subtable for group i'''
    group_table = Counter()
    for item in table.items():
        mult = item[0].count(i)
        if mult > 0:
            group_table.update({(i,)*mult : item[1]})
    return group_table

def plot_matrix_rows(matrix, title = 'Matrix rows'):
    max_mult = np.sum(np.sum(matrix,axis=0) > 0) - 1
    for i in xrange(n_groups):
        plt.semilogy(range(1, max_mult + 3), matrix[i,1:max_mult + 3],
                 label=i, color = plt.cm.Set1(i/n_groups))
    #plt.yscale('log')
    plt.ylim(np.min(matrix), 10 * np.max(matrix))
    plt.title(title)
    plt.legend(title='RG')
    plt.show() #change ticks to integers only

def plot_matrix(matrix, title='Matrix', vmax=None):
    n, m = matrix.shape
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.pcolor(matrix, edgecolor='k', cmap = plt.cm.Blues, vmin=0, vmax=vmax)
    plt.colorbar()
    ax.set_xticks(np.arange(n)+0.5)
    ax.set_yticks(np.arange(m)+0.5)
    ax.set_xticklabels(np.arange(n))
    ax.set_yticklabels(np.arange(m))
    ax.set_title(title)
    ax.invert_yaxis()
    plt.show()

def normalize_to_Markov(matrix):
    row_sums = matrix.sum(axis=1)
    return matrix / row_sums[:, np.newaxis]
    
def eigen(affinity):
    #change to computer for symmetrix matrix and convert?
    #is absolute value necessary? correct? won't fractoinal powers cause trouble?
    evals, evecs = np.linalg.eig(normalize_to_Markov(affinity))
    order = np.argsort(np.abs(evals))[::-1]
    return evals[order], evecs[:,order]

def diffusion_distances(evals, evecs, t = None):
    if not t:
        t = 1 / (1-evals[1])
    n_groups = len(evals)
    scaled_evecs = evecs[:,1:] * evals[1:]**t  #check that this works as expected
    dd_matrix = np.zeros((n_groups,n_groups))
    for i in xrange(n_groups):
        dd_matrix[i,i]=0
        for j in xrange(i+1, n_groups):
            difference = scaled_evecs[i,:] - scaled_evecs[j,:]
            dd_matrix[i,j] = np.sqrt(np.dot(difference,difference))
    return dd_matrix + dd_matrix.T

def plot_diffusion(evals, evecs, t=None, bound=None, title='Diffusion', color=None):
    if not t:
        t = 1 / (1-evals[1])
    x = evecs[:,1]*(evals[1]**t)
    y = evecs[:,2]*(evals[2]**t)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    lib_nums = library_nums(libraries)
    ax.scatter(x, y, color=color)
    ax.set_title(title)
    if bound:
        ax.set_xlim(-bound,bound)
        ax.set_ylim(-bound,bound)
    plt.show()

def plot_diffusion_3d(evals, evecs, t=None, bound=None, title='Diffusion', color=None):
    if not t:
        t = 1 / (1-evals[1])
    x = evecs[:,1]*(evals[1]**t)
    y = evecs[:,2]*(evals[2]**t)
    z = evecs[:,3]*(evals[3]**t)
    fig = plt.figure()
    ax = fig.add_subplot(111,projection="3d")
    ax.scatter3D(x, y, z, color = color)
    ax.set_title(title)
    if bound:
        ax.set_xlim3d(-bound,bound)
        ax.set_ylim3d(-bound,bound)
        ax.set_zlim3d(-bound,bound)
    plt.show()

def scatter_affinities(affinities, lib_matrices, pair_colors, log_scale = False, title='Scatter'):   
    fig = plt.figure(figsize=(10,8))
    xs = np.arange(len(pair_colors))
    plt.scatter(xs, affinities, s=2, color = pair_colors)
    plt.title(title)
    plt.ylim(1e-6,1)
    if log_scale == True:
        plt.yscale('log')
    for i in xs:
        if lib_matrices[i] == 3:
            plt.axvline(x=i,ls='-',alpha = .5,c='g',linewidth=1,)
    plt.show()
    
def plot_graph(matrix, threshold, color=None, complement = False):
    print 'Threshold =', threshold #include this in title
    n_groups = matrix.shape[0]
    labels = dict(zip(range(n_groups),range(n_groups)))
    if complement:
        G = nx.Graph(matrix < threshold)
    else:
        G = nx.Graph(matrix >= threshold)
    pos=nx.spring_layout(G)
    nx.draw_networkx_nodes(G,pos,node_color=color)
    nx.draw_networkx_edges(G,pos)
    nx.draw_networkx_labels(G,pos,labels=labels,font_color='w',font_size=16)
    plt.show()

In [1]:
#samples = ['G9166', 'G13337','C669','C322','C1688','C1700','C661','C1699','C1705','C1704','C1811']
#sample = 'C1700'
#sample = 'C1688'
sample = 'C1705'

groups, libs, n_reads, n_reads_removed, table = load_table(sample)
read_groups, libraries, table = remove_empty_groups(groups, libs, table)

n_groups = len(read_groups)
rg_sizes = group_sizes(table, n_groups)
lib_nums = library_nums(libraries)
n_libraries = np.max(lib_nums)
lib_matrix = library_matrix(lib_nums, n_groups)
lib_color = library_colors(lib_nums)
ol_affinity = overlap_affinity(table, n_groups)
mult_matrix = multiplicity_matrix(table, n_groups)
mult_matrix2 = normalize_rows_by_vector(mult_matrix, rg_sizes)
el2_affinity = L2_affinity(table, n_groups)
angle_matrix = np.arccos(el2_affinity)
ap_affinity = all_pairs_affinity(table, n_groups)
ap_affinity2 = all_pairs_affinity2(table, n_groups, rg_sizes)
ep_affinity = exact_pairs_affinity(table, n_groups)
ep_affinity2 = exact_pairs_affinity2(table, n_groups, rg_sizes)
evals1, evecs1 = eigen(el2_affinity)
evals2, evecs2 = eigen(ap_affinity)
dd_matrix1 = diffusion_distances(evals1,evecs1)
dd_matrix2 = diffusion_distances(evals2,evecs2)

print rg_sizes
print np.sum(rg_sizes)
print evals1
print evals2
plot_matrix_rows(mult_matrix, title='Number of n-tuples for each n by group')
#plot_matrix_rows(mult_matrix2, title='Number of n-tuples for each n by group, scaled')
plot_matrix(zero_diag(ol_affinity), title='Overlap affinity (Jacard distance)')
plot_matrix(zero_diag(el2_affinity), title='L2 affinity')
plot_matrix(zero_diag(ap_affinity), title='All pairs affinity')
#plot_matrix(zero_diag(ap_affinity2), title='All pairs affinity 2')
plot_matrix(zero_diag(ep_affinity), title='Exact pairs affinity')
#plot_matrix(zero_diag(ep_affinity2), title='Exact pairs affinity 2')
plot_diffusion(evals1,evecs1,title='L2 diffusion',color=lib_color)
plot_diffusion(evals2,evecs2,title='All pairs diffusion',color=lib_color)
plot_diffusion_3d(evals1,evecs1,title='L2 diffusion',color=lib_color)
plot_diffusion_3d(evals2,evecs2,title='All pairs diffusion',color=lib_color)
plot_graph(ap_affinity, .01, color=lib_color)
plot_graph(ap_affinity, .02, color=lib_color)
plot_graph(ap_affinity, .05, color=lib_color)
plot_graph(ap_affinity, .1, color=lib_color)
plot_graph(ap_affinity, .2, color=lib_color)
plot_graph(ap_affinity, .3, color=lib_color)
plot_graph(ap_affinity, .4, color=lib_color)
plot_graph(dd_matrix1, .001, color=lib_color, complement = True)
plot_graph(dd_matrix1, .003, color=lib_color, complement = True)
plot_graph(dd_matrix1, .01, color=lib_color, complement = True)
plot_graph(dd_matrix1, .03, color=lib_color, complement = True)
plot_graph(dd_matrix1, .1, color=lib_color, complement = True)
plot_graph(dd_matrix1, .3, color=lib_color, complement = True)
print read_groups
print libraries

#print multiplicity_vector(table)
#for item in table.items():
#    if len(item[0])>20:
#        print item

NameError: name 'load_table' is not defined

In [ ]:
samples = ['G9166', 'G13337', 'C669','C322','C1688','C1700','C661','C1699','C1705','C1704','C1811'] #removed C669
n_samples = len(samples)

ol_affinity_of_sample = {}
el2_affinity_of_sample = {}
ap_affinity_of_sample = {}
ep_affinity_of_sample = {}
lib_matrix_of_sample = {}
dd_matrix1_of_sample = {}
dd_matrix2_of_sample = {}

for sample in samples:
    groups, libs, n_reads, n_reads_removed, table = load_table(sample)
    read_groups, libraries, table = remove_empty_groups(groups, libs, table)
    n_groups = len(read_groups)
    rg_sizes = group_sizes(table, n_groups)
    lib_nums = library_nums(libraries)
    n_libraries = np.max(lib_nums)
    lib_color = library_colors(lib_nums)
    lib_matrix = library_matrix(lib_nums, n_groups)
    ol_affinity = overlap_affinity(table, n_groups)
    mult_matrix = multiplicity_matrix(table, n_groups)
    mult_matrix2 = normalize_rows_by_vector(mult_matrix, rg_sizes)
    el2_affinity = L2_affinity(table, n_groups)
    angle_matrix = np.arccos(el2_affinity)
    ap_affinity = all_pairs_affinity(table, n_groups)
    ap_affinity2 = all_pairs_affinity2(table, n_groups, rg_sizes)
    ep_affinity = exact_pairs_affinity(table, n_groups)
    ep_affinity2 = exact_pairs_affinity2(table, n_groups, rg_sizes)
    evals1, evecs1 = eigen(el2_affinity)
    evals2, evecs2 = eigen(ap_affinity)
    dd_matrix1 = diffusion_distances(evals1,evecs1)
    dd_matrix2 = diffusion_distances(evals2,evecs2)
    
    ol_affinity_of_sample[sample] = ol_affinity
    el2_affinity_of_sample[sample] = el2_affinity
    ap_affinity_of_sample[sample] = ap_affinity
    ep_affinity_of_sample[sample] = ep_affinity
    lib_matrix_of_sample[sample] = lib_matrix
    dd_matrix1_of_sample[sample] = dd_matrix1
    dd_matrix2_of_sample[sample] = dd_matrix2

In [ ]:
#ap_affinity_of_sample
ol_affinities = np.array([1,1,1])
el2_affinities = np.array([1,1,1])
ap_affinities = np.array([1,1,1])
ep_affinities = np.array([1,1,1])
dd_matrices1 = np.array([1,1,1])
dd_matrices2 = np.array([1,1,1])
lib_matrices = np.array([3,2,2])
for sample in samples:
    ol_affinity = ol_affinity_of_sample[sample]
    el2_affinity = el2_affinity_of_sample[sample]
    ap_affinity = ap_affinity_of_sample[sample]
    ep_affinity = ep_affinity_of_sample[sample]
    dd_matrix1 = dd_matrix1_of_sample[sample]
    dd_matrix2 = dd_matrix2_of_sample[sample]
    lib_matrix = lib_matrix_of_sample[sample]
    i_triu1 = np.triu_indices(ap_affinity.shape[0], k=1)
    ol_affinities = np.concatenate((ol_affinities, ol_affinity[i_triu1],np.array([1,1,1,1,1])))
    el2_affinities = np.concatenate((el2_affinities, el2_affinity[i_triu1],np.array([1,1,1,1,1])))
    ap_affinities = np.concatenate((ap_affinities, ap_affinity[i_triu1],np.array([1,1,1,1,1])))
    ep_affinities = np.concatenate((ep_affinities, ep_affinity[i_triu1],np.array([1,1,1,1,1])))
    dd_matrices1 = np.concatenate((dd_matrices1, dd_matrix1[i_triu1],np.array([1,1,1,1,1])))
    dd_matrices2 = np.concatenate((dd_matrices2, dd_matrix2[i_triu1],np.array([1,1,1,1,1])))
    lib_matrices = np.concatenate((lib_matrices, lib_matrix[i_triu1],np.array([2,2,3,2,2])))
    
def bool_to_color(x):
    if x == True:
        return 'b'
    elif x == False:
        return 'r'
    else:
        return 'w'

pair_colors = np.vectorize(bool_to_color)(lib_matrices)

In [ ]:
log_scale = False
scatter_affinities(ol_affinities, lib_matrices, pair_colors, title ='Overlap', log_scale=log_scale)
scatter_affinities(el2_affinities, lib_matrices, pair_colors, title ='L2', log_scale=log_scale)
scatter_affinities(ap_affinities, lib_matrices, pair_colors, title ='All pairs', log_scale=log_scale)
scatter_affinities(ep_affinities, lib_matrices, pair_colors, title ='Exact pairs', log_scale=log_scale)
scatter_affinities(dd_matrices1, lib_matrices, pair_colors, title ='Diffusion distance from L2', log_scale=log_scale)
scatter_affinities(dd_matrices2, lib_matrices, pair_colors, title ='Diffusion distance from all pairs', log_scale=log_scale)

In [ ]:
plt.hist(ap_affinities, bins=np.arange(0,1,.001))
plt.show()

In [ ]:
#Investigate empirical and expected distribution of multiplicites
#e.g., how many values occur i times for each i
#Here we assume a uniform distribution, later should try less entropy

#set and advance seed
try:
    seed += 1
except:
    seed = 0
rs = np.random.RandomState(seed)

n_dist = 10000000
n_sample = 3684904

sample = rs.multinomial(n_sample, np.ones(n_dist)/n_dist)
max_mult = np.max(sample)
mult_array_emp = np.array([np.sum(sample == i) for i in range(max_mult+1)])
mult_array_exp = n_dist * ss.binom(n_sample, 1/n_dist).pmf(np.arange(max_mult+1))
#def mult_exp(n_dist, n_sample, i):  #this is equivalent
#    return comb(n_sample,i) * (1/n_dist)**(i-1) * (1 - 1/n_dist)**(n_sample - i)
#mult_array_exp = np.array([n_dist * (1-1/n_dist)**n_sample] + [mult_exp(n_dist, n_sample, i) for i in range(1,max_mult+1)])

np.set_printoptions(precision=1)

print mult_array_emp
print mult_array_exp

#plt.scatter(range(max_mult + 1), mult_array_emp)
plt.scatter(range(max_mult + 1), mult_array_exp, color = 'r')
plt.yscale('log')
plt.ylim(.1,10*max(mult_array_emp))
plt.title('Number of n-tuples for each n')
plt.show()

In [ ]:
np.set_printoptions(precision = 6)
print ap_affinity / el2_affinity #quite different
print ap_affinity2 / el2_affinity #very similar, only difference is that size of read group is not same as length of read group'

In [ ]:
print ap_affinity
print el2_affinity

In [ ]:
for library in libraries:
    print library

for group in groups:
    print group

In [ ]:
print all_pairs.__doc__
print all_pairs.__name__
print all_pairs.__defaults__
print all_pairs.__code__
#print all_pairs.__globals__
print all_pairs.__dict__
print all_pairs.__closure__

print plot_matrix.__doc__
print plot_matrix.__name__
print plot_matrix.__defaults__
print plot_matrix.__code__
#print plot_matrix.__globals__
print plot_matrix.__dict__
print plot_matrix.__closure__

In [ ]:
np.set_printoptions(precision=1, suppress=True)

print all_pairs(table, n_groups)
print inner_product(table, n_groups)

In [ ]:
!ls -lathr

In [ ]:
len(table.keys())

In [ ]:
table

In [ ]:
len(table.keys())

Bradscratch stuff below here

In [ ]:
# Source data from printouts. I don't have the original data
table = {(5,): 140358, (3,): 139360, (1,): 124943, (4,): 123598, (2,): 122481, (0,): 108269, (5, 5): 13190, (3, 3): 12908, (1, 1): 11659, (4, 4): 11056, (2, 2): 10878, (0, 0): 9780, (3, 5): 5807, (1, 3): 5595, (2, 4): 5338, (0, 2): 5116, (1, 5): 2240, (0, 4): 2197, (3, 3, 3): 1617, (5, 5, 5): 1604, (1, 1, 1): 1449, (4, 4, 4): 1297, (2, 2, 2): 1253, (3, 3, 5): 1148, (0, 0, 0): 1143, (3, 5, 5): 1127, (1, 3, 3): 1066, (2, 4, 4): 1027, (1, 1, 3): 992, (2, 2, 4): 974, (0, 2, 2): 968, (1, 3, 5): 932, (0, 2, 4): 929, (0, 0, 2): 855, (2, 3): 563, (4, 5): 522, (0, 1): 488, (1, 5, 5): 446, (0, 4, 4): 438, (0, 0, 4): 430, (1, 1, 5): 368, (1, 3, 3, 5): 310, (3, 3, 5, 5): 267, (1, 3, 5, 5): 241, (5, 5, 5, 5): 240, (2, 2, 4, 4): 235, (1, 1, 3, 3): 233, (3, 3, 3, 5): 232, (0, 2, 2, 4): 228, (0, 0, 2, 4): 212, (3, 3, 3, 3): 209, (1, 1, 1, 1): 207, (0, 2, 4, 4): 205, (3, 5, 5, 5): 204, (1, 1, 3, 5): 202, (1, 3, 3, 3): 195, (0, 0, 2, 2): 179, (1, 1, 1, 3): 178, (4, 4, 4, 4): 175, (2, 4, 4, 4): 172, (0, 2, 2, 2): 172, (2, 2, 2, 4): 165, (2, 2, 2, 2): 155, (0, 0, 0, 0): 140, (0, 0, 0, 2): 137, (1, 1, 5, 5): 110, (0, 0, 4, 4): 96, (4, 5, 5): 72, (1, 3, 3, 5, 5): 71, (0, 1, 1): 69, (0, 0, 2, 2, 4): 69, (1, 1, 3, 3, 5): 64, (0, 4, 4, 4): 64, (2, 3, 3): 63, (0, 0, 2, 4, 4): 62, (3, 3, 3, 5, 5): 61, (1, 5, 5, 5): 61, (0, 0, 0, 4): 58, (3, 3, 5, 5, 5): 57, (1, 1, 3, 5, 5): 57, (0, 2, 2, 4, 4): 55, (2, 2, 3): 53, (4, 4, 5): 52, (0, 2, 2, 2, 4): 51, (1, 3, 5, 5, 5): 51, (3, 4): 51, (1, 1, 1, 3, 3): 51, (2, 5): 48, (1, 3, 3, 3, 5): 47, (0, 0, 1): 47, (0, 3): 46, (1, 1, 1, 5): 45, (1, 1, 3, 3, 3): 45, (1, 3, 3, 3, 3): 43, (0, 0, 0, 2, 4): 42, (2, 2, 4, 4, 4): 42, (1, 2): 42, (1, 1, 1, 3, 5): 41, (2, 2, 2, 4, 4): 39, (3, 5, 5, 5, 5): 38, (0, 0, 0, 2, 2): 37, (0, 2, 4, 4, 4): 37, (3, 3, 3, 3, 3): 36, (0, 2, 2, 2, 2): 30, (3, 3, 3, 3, 5): 30, (5, 5, 5, 5, 5): 29, (2, 4, 4, 4, 4): 27, (0, 0, 2, 2, 2): 26, (1, 1, 1, 1, 3): 26, (0, 0, 0, 0, 0): 24, (0, 0, 4, 4, 4): 24, (0, 0, 0, 0, 2): 23, (4, 4, 4, 4, 4): 23, (1, 1, 1, 1, 1): 22, (2, 2, 2, 2, 2): 22, (1, 1, 3, 5, 5, 5): 22, (2, 2, 2, 2, 4): 21, (1, 1, 3, 3, 5, 5): 20, (1, 1, 3, 3, 3, 5): 19, (1, 1, 1, 1, 3, 3): 19, (3, 4, 5): 18, (0, 0, 2, 2, 4, 4): 17, (1, 1, 1, 3, 5, 5): 17, (1, 2, 3): 17, (1, 3, 3, 3, 5, 5): 16, (1, 1, 1, 5, 5): 16, (1, 1, 5, 5, 5): 15, (0, 2, 2, 4, 4, 4): 15, (0, 2, 2, 2, 4, 4): 14, (1, 3, 3, 5, 5, 5): 14, (0, 0, 2, 2, 2, 4): 14, (3, 3, 3, 5, 5, 5): 14, (3, 3, 5, 5, 5, 5): 14, (2, 3, 5): 13, (0, 0, 0, 4, 4): 13, (1, 1, 1, 3, 3, 5): 12, (2, 4, 5): 12, (2, 3, 4): 12, (0, 1, 2): 12, (4, 5, 5, 5): 12, (3, 3, 3, 3, 5, 5): 11, (2, 2, 3, 3): 11, (1, 1, 1, 3, 3, 3): 11, (0, 0, 2, 4, 4, 4): 11, (0, 0, 2, 2, 2, 2): 11, (1, 3, 3, 3, 5, 5, 5): 10, (1, 3, 3, 3, 3, 5): 10, (1, 5, 5, 5, 5): 10, (1, 1, 1, 1, 5): 10, (1, 1, 1, 1, 3, 5): 10, (0, 1, 1, 1): 10, (1, 1, 3, 3, 3, 3): 9, (0, 4, 4, 4, 4): 9, (0, 2, 3): 9, (2, 2, 2, 3): 9, (0, 0, 0, 2, 4, 4): 9, (0, 2, 2, 2, 2, 4): 8, (0, 1, 2, 2): 8, (5, 5, 5, 5, 5, 5): 8, (4, 4, 5, 5): 8, (1, 3, 3, 3, 3, 3): 8, (4, 4, 4, 5): 8, (0, 1, 3): 8, (0, 0, 0, 2, 2, 4, 4): 7, (0, 1, 2, 3): 7, (0, 0, 0, 0, 2, 2): 7, (2, 2, 2, 4, 4, 4): 7, (1, 1, 3, 3, 5, 5, 5): 7, (1, 2, 2): 7, (0, 0, 0, 1): 7, (0, 0, 0, 2, 2, 2): 7, (0, 0, 0, 2, 2, 4): 7, (0, 0, 0, 0, 4): 7, (0, 0, 0, 0, 2, 4): 7, (1, 4): 7, (1, 3, 5, 5, 5, 5): 7, (0, 2, 2, 2, 4, 4, 4): 6, (1, 1, 1, 3, 5, 5, 5): 6, (2, 2, 5): 6, (1, 1, 1, 1, 3, 5, 5): 6, (0, 2, 4, 4, 4, 4): 6, (3, 3, 4): 6, (2, 5, 5): 6, (1, 1, 1, 1, 1, 3): 6, (2, 2, 2, 2, 4, 4): 6, (0, 0, 1, 1): 6, (1, 1, 1, 1, 1, 3, 3): 5, (1, 1, 3, 5, 5, 5, 5): 5, (0, 0, 0, 0, 0, 2): 5, (1, 1, 1, 1, 1, 1): 5, (0, 0, 3): 5, (0, 0, 0, 0, 2, 2, 4): 5, (1, 1, 5, 5, 5, 5): 5, (2, 4, 4, 5): 5, (0, 0, 0, 0, 4, 4): 5, (1, 1, 1, 3, 3, 3, 3): 5, (1, 1, 2): 5, (3, 4, 4): 5, (0, 0, 2, 2, 2, 4, 4, 4): 4, (1, 1, 1, 3, 3, 5, 5): 4, (0, 5): 4, (1, 2, 5): 4, (1, 1, 1, 5, 5, 5): 4, (2, 3, 3, 5, 5): 4, (2, 3, 5, 5): 4, (3, 5, 5, 5, 5, 5): 4, (4, 4, 4, 4, 4, 4): 4, (3, 3, 3, 3, 5, 5, 5): 4, (0, 2, 2, 3): 4, (1, 1, 3, 3, 3, 5, 5): 4, (5, 5, 5, 5, 5, 5, 5): 4, (0, 5, 5): 3, (0, 2, 2, 2, 2, 2, 4): 3, (0, 2, 5): 3, (0, 0, 0, 4, 4, 4): 3, (0, 0, 4, 4, 4, 4): 3, (3, 3, 3, 5, 5, 5, 5): 3, (1, 1, 3, 3, 3, 5, 5, 5): 3, (2, 2, 2, 2, 4, 4, 4): 3, (2, 3, 3, 5): 3, (0, 3, 3): 3, (0, 2, 2, 2, 2, 2, 4, 4): 3, (0, 0, 2, 2, 4, 4, 4): 3, (2, 2, 2, 2, 2, 2): 3, (1, 1, 3, 3, 3, 3, 5): 3, (3, 3, 4, 5): 3, (0, 0, 0, 2, 2, 4, 4, 4): 3, (1, 3, 3, 5, 5, 5, 5): 3, (1, 3, 3, 3, 3, 3, 5): 3, (2, 2, 3, 4): 3, (0, 1, 3, 3): 3, (1, 1, 1, 3, 3, 3, 5, 5): 3, (1, 1, 1, 1, 3, 3, 5): 3, (1, 3, 3, 3, 3, 3, 3): 3, (1, 1, 3, 3, 3, 3, 3): 3, (2, 3, 3, 3): 3, (2, 4, 4, 4, 4, 4): 3, (1, 3, 3, 3, 3, 5, 5): 3, (2, 2, 4, 4, 4, 4): 3, (1, 4, 4): 2, (4, 5, 5, 5, 5): 2, (1, 3, 3, 5, 5, 5, 5, 5): 2, (3, 3, 3, 3, 3, 5): 2, (4, 4, 4, 5, 5): 2, (1, 1, 1, 1, 1, 3, 5, 5): 2, (0, 0, 0, 0, 0, 4): 2, (3, 4, 4, 5): 2, (2, 2, 3, 3, 3): 2, (2, 2, 3, 5): 2, (0, 0, 2, 2, 2, 2, 4, 4): 2, (1, 1, 4): 2, (0, 2, 3, 3): 2, (1, 1, 1, 1, 3, 3, 5, 5, 5): 2, (1, 1, 1, 3, 5, 5, 5, 5): 2, (2, 2, 2, 2, 2, 2, 4): 2, (0, 0, 0, 0, 0, 2, 4): 2, (0, 0, 2, 2, 2, 4, 4): 2, (0, 0, 0, 0, 2, 2, 2, 2): 2, (1, 1, 1, 3, 3, 3, 3, 5, 5): 2, (2, 2, 2, 2, 2, 4): 2, (0, 1, 1, 3): 2, (0, 2, 2, 2, 2, 4, 4, 4): 2, (0, 0, 0, 0, 2, 2, 4, 4): 2, (4, 4, 5, 5, 5): 2, (3, 4, 4, 4): 2, (2, 3, 3, 4, 5): 2, (0, 3, 3, 5): 2, (0, 2, 3, 4): 2, (2, 3, 4, 5, 5): 2, (1, 1, 1, 1, 1, 5): 2, (0, 0, 0, 2, 2, 2, 4): 2, (0, 0, 2, 2, 2, 2, 2): 2, (1, 1, 1, 1, 3, 5, 5, 5): 2, (1, 3, 5, 5, 5, 5, 5): 2, (0, 0, 2, 2, 2, 2, 2, 4): 2, (0, 0, 0, 0, 2, 4, 4): 2, (0, 0, 0, 1, 2): 2, (3, 3, 3, 3, 3, 3, 5): 2, (3, 3, 3, 3, 3, 3): 2, (3, 3, 3, 3, 3, 5, 5): 2, (0, 2, 2, 2, 2, 2, 2): 2, (1, 2, 3, 3): 2, (0, 0, 1, 3): 2, (1, 1, 1, 1, 5, 5): 2, (0, 0, 0, 2, 2, 2, 2): 2, (2, 3, 3, 4): 2, (3, 3, 3, 4): 2, (1, 1, 2, 3): 2, (3, 3, 3, 3, 3, 3, 3): 2, (0, 0, 2, 2, 2, 2, 4): 2, (0, 0, 1, 2): 2, (1, 3, 3, 4): 2, (1, 1, 1, 2): 2, (0, 0, 0, 2, 4, 4, 4): 2, (1, 1, 1, 1, 1, 1, 3): 2, (1, 1, 3, 3, 3, 3, 3, 5, 5): 2, (2, 4, 5, 5, 5): 2, (0, 0, 0, 0, 2, 2, 2): 2, (3, 3, 5, 5, 5, 5, 5): 2, (0, 0, 0, 2, 2, 2, 2, 4): 1, (2, 3, 3, 5, 5, 5, 5, 5): 1, (1, 1, 3, 3, 3, 3, 5, 5): 1, (0, 0, 2, 2, 2, 3): 1, (2, 3, 3, 3, 3, 4, 5): 1, (0, 0, 2, 2, 3): 1, (0, 1, 1, 3, 3, 3, 5): 1, (0, 2, 3, 3, 5): 1, (2, 2, 2, 4, 5): 1, (1, 2, 4): 1, (0, 0, 2, 2, 2, 2, 2, 4, 4): 1, (2, 2, 4, 4, 4, 4, 4): 1, (0, 1, 2, 2, 2, 3): 1, (0, 1, 1, 2): 1, (1, 1, 1, 1, 1, 3, 3, 5): 1, (0, 1, 1, 5): 1, (3, 3, 3, 4, 4): 1, (0, 1, 1, 1, 1, 1): 1, (0, 0, 1, 4): 1, (0, 0, 0, 1, 1, 1, 3, 3): 1, (2, 2, 3, 4, 5): 1, (1, 3, 3, 3, 5, 5, 5, 5): 1, (1, 4, 4, 5, 5): 1, (1, 1, 2, 5, 5): 1, (0, 2, 3, 3, 3): 1, (1, 1, 1, 1, 3, 3, 3, 5): 1, (3, 3, 3, 4, 5): 1, (0, 2, 2, 2, 2, 2): 1, (1, 1, 1, 3, 3, 3, 5, 5, 5): 1, (1, 1, 1, 1, 5, 5, 5): 1, (3, 4, 5, 5, 5): 1, (1, 1, 2, 4, 5, 5): 1, (1, 1, 1, 3, 3, 3, 3, 3, 5): 1, (0, 0, 0, 3): 1, (2, 3, 4, 4, 4): 1, (4, 4, 4, 4, 5): 1, (1, 2, 2, 2, 2): 1, (1, 1, 2, 2, 4): 1, (1, 2, 3, 4, 4): 1, (3, 3, 3, 3, 5, 5, 5, 5): 1, (2, 3, 4, 5): 1, (0, 0, 0, 2, 2, 2, 2, 2): 1, (2, 3, 3, 3, 4, 5): 1, (1, 1, 3, 3, 3, 3, 3, 5, 5, 5): 1, (0, 1, 1, 2, 2): 1, (1, 1, 1, 1, 1, 5, 5): 1, (3, 4, 4, 5, 5, 5): 1, (1, 1, 3, 3, 5, 5, 5, 5): 1, (0, 2, 3, 4, 5): 1, (0, 0, 2, 3, 3, 4, 5, 5, 5): 1, (2, 2, 2, 2, 2, 4, 4, 4): 1, (0, 0, 0, 0, 0, 0, 0): 1, (1, 3, 3, 3, 4, 5): 1, (2, 2, 5, 5): 1, (0, 1, 2, 2, 3, 3): 1, (0, 4, 4, 4, 4, 4): 1, (0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5): 1, (0, 0, 0, 0, 2, 4, 4, 4): 1, (1, 2, 2, 3): 1, (0, 2, 4, 4, 4, 4, 4): 1, (0, 3, 3, 3): 1, (0, 0, 0, 0, 0, 0): 1, (0, 0, 2, 5): 1, (1, 1, 3, 3, 3, 5, 5, 5, 5): 1, (3, 4, 4, 4, 4): 1, (0, 1, 4, 5): 1, (1, 1, 3, 4, 5): 1, (0, 0, 0, 2, 2, 2, 2, 2, 4): 1, (1, 3, 3, 3, 3, 3, 5, 5): 1, (1, 2, 3, 4): 1, (0, 0, 1, 2, 2): 1, (3, 3, 4, 4): 1, (1, 1, 3, 3, 3, 3, 3, 5): 1, (0, 0, 2, 2, 2, 4, 4, 4, 4): 1, (1, 1, 1, 3, 3, 3, 3, 5): 1, (0, 1, 1, 3, 5): 1, (0, 1, 3, 3, 3): 1, (0, 0, 0, 0, 0, 2, 4, 4): 1, (0, 0, 2, 2, 2, 2, 2, 3, 3, 4): 1, (3, 3, 3, 5, 5, 5, 5, 5, 5): 1, (0, 0, 1, 2, 3): 1, (0, 0, 1, 1, 3): 1, (0, 0, 2, 2, 4, 4, 4, 4, 4): 1, (1, 1, 1, 3, 3, 3, 5): 1, (0, 0, 2, 2, 3, 4): 1, (0, 0, 2, 4, 4, 5, 5): 1, (1, 3, 3, 3, 5, 5, 5, 5, 5): 1, (3, 3, 4, 5, 5, 5): 1, (2, 2, 2, 2, 2, 2, 2, 4): 1, (0, 0, 0, 0, 4, 4, 4): 1, (1, 1, 2, 3, 5, 5): 1, (0, 3, 4, 5): 1, (2, 3, 3, 4, 4): 1, (2, 2, 3, 4, 4, 5): 1, (0, 0, 0, 0, 0, 2, 2): 1, (1, 1, 1, 1, 3, 3, 3, 3): 1, (0, 1, 2, 2, 2, 2): 1, (1, 3, 5, 5, 5, 5, 5, 5): 1, (0, 0, 0, 0, 1): 1, (2, 3, 3, 3, 3, 3, 3, 5, 5): 1, (0, 0, 0, 2, 4, 4, 4, 4, 4): 1, (2, 3, 4, 4): 1, (1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 5, 5): 1, (3, 4, 5, 5): 1, (1, 3, 3, 3, 3, 5, 5, 5, 5): 1, (2, 2, 3, 3, 3, 4, 5, 5): 1, (0, 2, 2, 4, 4, 4, 4, 4, 4, 4): 1, (0, 0, 1, 1, 4): 1, (0, 0, 1, 2, 5): 1, (1, 1, 1, 1, 1, 1, 1): 1, (0, 0, 2, 2, 2, 3, 4, 4, 5): 1, (1, 2, 2, 2, 2, 2, 2, 3): 1, (2, 2, 2, 3, 3): 1, (0, 3, 5): 1, (2, 2, 2, 2, 2, 2, 2): 1, (0, 1, 1, 2, 3, 5): 1, (1, 1, 1, 1, 1, 1, 3, 3): 1, (1, 1, 1, 1, 3, 3, 5, 5): 1, (0, 0, 1, 2, 2, 4): 1, (1, 1, 1, 1, 1, 3, 5): 1, (1, 1, 3, 4): 1, (2, 4, 5, 5): 1, (0, 0, 1, 1, 1, 1): 1, (3, 3, 3, 4, 4, 4, 5): 1, (1, 5, 5, 5, 5, 5): 1, (3, 3, 5, 5, 5, 5, 5, 5, 5): 1, (1, 2, 3, 5): 1, (3, 5, 5, 5, 5, 5, 5): 1, (0, 0, 1, 1, 2, 3): 1, (2, 2, 2, 2, 3, 4, 5): 1, (0, 0, 0, 0, 0, 2, 2, 2, 2, 2): 1, (1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 5): 1, (0, 0, 1, 2, 3, 3, 5): 1, (0, 0, 2, 2, 2, 2, 4, 4, 4, 4): 1, (2, 2, 3, 4, 5, 5, 5): 1, (0, 0, 0, 1, 2, 3): 1, (0, 1, 1, 1, 1): 1, (1, 2, 4, 5): 1, (1, 1, 1, 1, 1, 1, 3, 3, 3): 1, (1, 1, 1, 1, 1, 1, 1, 3): 1, (0, 0, 2, 2, 2, 2, 4, 4, 4): 1, (3, 3, 3, 3, 5, 5, 5, 5, 5): 1, (2, 4, 4, 4, 5, 5): 1, (1, 1, 1, 5, 5, 5, 5): 1, (0, 0, 1, 1, 1): 1, (0, 0, 0, 1, 1, 1): 1, (0, 1, 1, 3, 3, 4, 5): 1, (0, 0, 0, 0, 2, 2, 2, 4): 1, (4, 4, 4, 4, 4, 4, 5): 1, (0, 0, 2, 4, 4, 4, 4): 1, (0, 1, 1, 1, 2): 1, (0, 2, 2, 2, 2, 4, 4): 1, (0, 3, 4, 4): 1, (0, 2, 2, 2, 3): 1, (2, 2, 2, 2, 2, 4, 4): 1, (1, 1, 1, 1, 2, 4, 5): 1, (0, 0, 0, 0, 0, 0, 2, 2, 2, 4): 1, (0, 0, 1, 1, 2): 1, (2, 2, 2, 2, 2, 5): 1, (1, 1, 1, 5, 5, 5, 5, 5): 1, (0, 0, 0, 2, 2, 2, 4, 4): 1, (0, 0, 0, 0, 2, 2, 2, 4, 4): 1}
sample = 'C1705'
read_groups = ['H8VFV.1', 'H8VFV.1.1', 'H8VJG.1', 'H8VJG.1.3', 'H8VJG.2', 'H8VJG.2.5']
libraries = ['Pond-342622', 'Pond-342631', 'Pond-342622', 'Pond-342631', 'Pond-342622', 'Pond-342631']

# john's stuff
n_groups = len(read_groups)
rg_sizes = group_sizes(table, n_groups)
lib_nums = library_nums(libraries)
n_libraries = np.max(lib_nums)
lib_matrix = library_matrix(lib_nums, n_groups)
lib_color = library_colors(lib_nums)
ol_affinity = overlap_affinity(table, n_groups)
mult_matrix = multiplicity_matrix(table, n_groups)
mult_matrix2 = normalize_rows_by_vector(mult_matrix, rg_sizes)
el2_affinity = L2_affinity(table, n_groups)
angle_matrix = np.arccos(el2_affinity)
ap_affinity = all_pairs_affinity(table, n_groups)
ap_affinity2 = all_pairs_affinity2(table, n_groups, rg_sizes)
ep_affinity = exact_pairs_affinity(table, n_groups)
ep_affinity2 = exact_pairs_affinity2(table, n_groups, rg_sizes)
evals1, evecs1 = eigen(el2_affinity)
evals2, evecs2 = eigen(ap_affinity)
dd_matrix1 = diffusion_distances(evals1,evecs1)
dd_matrix2 = diffusion_distances(evals2,evecs2)

print "ap_affinity"
print ap_affinity
print "sums"
row_sums = ap_affinity.sum(axis=1)
print row_sums
print "row_sums[:, np.newaxis]"
denom = row_sums[:, np.newaxis]
print denom
print "num/denom"
print ap_affinity / denom
print "function"
print normalize_to_Markov(ap_affinity)

In [ ]:
%matplotlib inline
#%time

import json

import numpy as np
import pandas as pd
import networkx as nx
from scipy.misc import comb
from scipy.special import binom
import scipy.stats as ss
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.mplot3d import Axes3D
from collections import Counter
from __future__ import division

def load_table(sample):
    f = open(sample + '.table.txt', 'r')
    read_groups = eval(f.readline())
    libraries = eval(f.readline())
    n_reads = eval(f.readline())
    n_reads_removed = eval(f.readline())
    table = eval(f.readline())
    f.close()
    return read_groups, libraries, n_reads, n_reads_removed, table

def group_sizes(table, n_groups):
    '''Returns vector v where v[i] is count of reads in group i.'''
    rg_sizes = np.zeros(n_groups, dtype=int)
    for item in table.items():
        for i in item[0]:
            rg_sizes[i] += item[1]
    return rg_sizes

def remove_empty_groups(read_groups, libraries, table):
    n_groups = len(read_groups)
    rg_sizes = group_sizes(table, n_groups)
    if np.prod(rg_sizes != 0):
        return read_groups, libraries, table
    else:
        groups = [read_groups[i] for i in range(n_groups) if rg_sizes[i] != 0]
        libs = [libraries[i] for i in range(n_groups) if rg_sizes[i] != 0]
        new_num_of_old_num = {}
        current_num = 0
        for i in range(n_groups):
            new_num_of_old_num[i] = current_num
            if rg_sizes[i] != 0:
                current_num += 1
        new_table = Counter()
        for item in table.items():
            new_key = tuple(map(lambda x : new_num_of_old_num[x], list(item[0])))
            new_table.update({new_key : item[1]})
        return groups, libs, new_table
    
def library_nums(libraries):
    n_groups = len(libraries)
    num_of_library = {}
    current_num = 0
    for i in xrange(n_groups):
        if libraries[i] not in num_of_library.keys(): #change to set_default
            num_of_library[libraries[i]] = current_num
            current_num += 1
    return np.array([num_of_library[libraries[i]] for i in xrange(n_groups)])

def library_colors(lib_nums):
    #lib_color = plt.cm.Set1(lib_nums / (n_libraries + 1))
    color_of_num = {0 : 'r', 1 : 'b', 2 : 'g', 3 : 'k', 4 : 'y', 5 : 'm', 6 : 'c'}
    return [color_of_num[lib_num % 5] for lib_num in lib_nums]

def library_matrix(lib_nums, n_groups):
    lib_matrix = np.zeros((n_groups,n_groups), dtype=bool)
    for i in xrange(n_groups):
        for j in xrange(n_groups):
            lib_matrix[i,j] = (lib_nums[i] == lib_nums[j])
    return lib_matrix

# jaccard similarity. union/intersection. might not be robust to diff conditions. no a priori expectation
def overlaps(table, n_groups):
    '''Returns matrix where M[i][j] is count of read types in both groups i
       and group j (symmetrized)'''
    overlap_matrix = np.zeros((n_groups, n_groups))
    for item in table.items():
        groups = list(set(item[0]))
        l = len(groups)
        for i in xrange(0,l):
            for j in xrange(i,l):
                overlap_matrix[groups[i],groups[j]] += item[1]
    return (overlap_matrix + overlap_matrix.T) / 2

def inner_product(table, n_groups):
    '''Returns matrix X^t * X where X[i,j] is count of reads of type i in group j.
       Agrees with all_pairs off the diagonal.'''
    ip_matrix = np.zeros((n_groups,n_groups))
    for item in table.items():
        l = len(item[0])
        count_vector = np.zeros(n_groups)
        for i in xrange(l):
            count_vector[item[0][i]] += 1
        ip_matrix += np.outer(count_vector, count_vector) * item[1]
    return ip_matrix


# ABB would be 3 pairs- AB, AB, BB
# Retain signal from non-exact pairs. didn't do this as the only metric bc seeing a lot of on-flowcell pad-hopping duplicates, showing up as large duplicate sets, which would dominate here
# We only want biological duplicates, and we have been better at marking these. The work yossi and david benj are doing. What's going on from david roazen.
# That wasn't ready yet, so he did an exact pairs approach as well.
# This one is probably best if you are careful to look at only biological duplicates
def all_pairs(table, n_groups):
    '''Returns matrix where M[i,j] is count of matching pairs of distinct reads
       from groups i and j (symmetrized).'''
    ap_matrix = np.zeros((n_groups,n_groups))
    for item in table.items():
        l = len(item[0])
        for i in xrange(l-1):
            for j in xrange(i+1,l):
                ap_matrix[item[0][i],item[0][j]] += item[1]
    return (ap_matrix + np.tril(ap_matrix.T))/2

def exact_tuples(table, n_groups, k):
    '''Returns array where A[i_1,...i_k] is count of read types that occur
       exactly in the groups i_1,...,i_k ascending with repeats.'''
    et_array = np.zeros(k*(n_groups,))
    for item in table.items():
        if len(item[0])==k:
            et_array[item[0]] = item[1]
    return et_array

def exact_pairs(table, n_groups):
    '''Returns matrix where M[i,j] is count of read types that occur exactly twice,
       once in group i and once in group j (symmetrized).'''
    ep_matrix = exact_tuples(table, n_groups, 2)
    return (ep_matrix + np.tril(ep_matrix.T))/2

def normalize(array):
    return array/np.sum(array)

# normalize such that diagonals are 1 (so 1 is max self-similarity). off-diag things scaled proportionally
def normalize_by_diagonal(matrix):
    diag = np.diagonal(matrix)
    return matrix / np.sqrt(np.outer(diag, diag))

# take the outer product of [pq] with itself
# ie [[p^2, pq], [pq, q^2]]
# so divide matrix by [[p, rad(pq)], [rad(pq), q]]
# so each cell is divided by the total number of elements in it's thing.
# most similar to the underlying model
def normalize_by_group_size(matrix, group_sizes):
    return matrix / np.sqrt(np.outer(group_sizes, group_sizes))

def normalize_rows_by_vector(matrix, vector):
    return matrix / vector[:, np.newaxis]

def overlap_affinity(table, n_groups):
    ol_matrix = overlaps(table, n_groups)
    ol_affinity = np.zeros((n_groups, n_groups))
    for i in xrange(n_groups):
        ol_affinity[i,i] = .5
        for j in xrange(i+1,n_groups):
            ol_affinity[i,j] = 2 * ol_matrix[i,j] / \
                               (ol_matrix[i,i] + ol_matrix[j,j] - 2 * ol_matrix[i,j])
    return ol_affinity + ol_affinity.T

def exact_pairs_affinity(table, n_groups):
    return normalize_by_diagonal(exact_pairs(table, n_groups))

def all_pairs_affinity(table, n_groups):
    return normalize_by_diagonal(all_pairs(table, n_groups))

# Distance in euclidean space between 2 vectrs. each RG is a vector in a space whose dimension is the number of non-zero entries in the union of observed inserts
# so, what's teh cosine fo the angle between these 2 vectors
# naturally, the diagonal elements will be 1
# cosine of angle = (A dot B) / (length A)(length B)

# BUT our data isn't in the form of these vectors. So we count the AB pairs.
# Recording each time he sees some combination of 2 numbers.

# suffers the same issue as jaccard- what is your expectation. this is why all pairs and exact pairs only viable.
def L2_affinity(table, n_groups):
    return normalize_by_diagonal(inner_product(table, n_groups))

def exact_pairs_affinity2(table, n_groups, rg_sizes):
    return normalize(normalize_by_group_size(exact_pairs(table, n_groups),rg_sizes))

def all_pairs_affinity2(table, n_groups, rg_sizes):
    return normalize(normalize_by_group_size(all_pairs(table, n_groups),rg_sizes))

def zero_diag(matrix):
    new_matrix = matrix.copy()
    for i in xrange(matrix.shape[0]):
        new_matrix[i,i] = 0
    return new_matrix

def multiplicity_matrix(table, n_groups):
    '''Returns M with M_ij count of j-tuples in group i'''
    max_mult =  max(map(len,table.keys()))
    mult_matrix = np.zeros((n_groups, max_mult + 1), dtype=int)
    for i in xrange(n_groups):
        mult_vector = multiplicity_vector(subtable(table,i))
        for j in xrange(len(mult_vector)):
            mult_matrix[i,j] = mult_vector[j]
    return mult_matrix

def multiplicity_vector(table):
    '''Returns v with v_j count of j-tuples in table'''
    max_mult =  max(map(len,table.keys()))
    mult_vector = np.zeros(max_mult + 1, dtype=int)
    for item in table.items():
        mult_vector[len(item[0])] += item[1]   
    return mult_vector

def subtable(table, i):
    '''Returns subtable for group i'''
    group_table = Counter()
    for item in table.items():
        mult = item[0].count(i)
        if mult > 0:
            group_table.update({(i,)*mult : item[1]})
    return group_table

def plot_matrix_rows(matrix, title = 'Matrix rows'):
    max_mult = np.sum(np.sum(matrix,axis=0) > 0) - 1
    for i in xrange(n_groups):
        plt.semilogy(range(1, max_mult + 3), matrix[i,1:max_mult + 3],
                 label=i, color = plt.cm.Set1(i/n_groups))
    #plt.yscale('log')
    plt.ylim(np.min(matrix), 10 * np.max(matrix))
    plt.title(title)
    plt.legend(title='RG')
    plt.show() #change ticks to integers only

def plot_matrix(matrix, title='Matrix', vmax=None):
    n, m = matrix.shape
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.pcolor(matrix, edgecolor='k', cmap = plt.cm.Blues, vmin=0, vmax=vmax)
    plt.colorbar()
    ax.set_xticks(np.arange(n)+0.5)
    ax.set_yticks(np.arange(m)+0.5)
    ax.set_xticklabels(np.arange(n))
    ax.set_yticklabels(np.arange(m))
    ax.set_title(title)
    ax.invert_yaxis()
    plt.show()

def normalize_to_Markov(matrix):
    row_sums = matrix.sum(axis=1)
    return matrix / row_sums[:, np.newaxis]
    
def eigen(affinity):
    #change to computer for symmetrix matrix and convert?
    #is absolute value necessary? correct? won't fractoinal powers cause trouble?
    evals, evecs = np.linalg.eig(normalize_to_Markov(affinity))
    order = np.argsort(np.abs(evals))[::-1]
    return evals[order], evecs[:,order]

def diffusion_distances(evals, evecs, t = None):
    if not t:
        t = 1 / (1-evals[1])
    n_groups = len(evals)
    scaled_evecs = evecs[:,1:] * evals[1:]**t  #check that this works as expected
    dd_matrix = np.zeros((n_groups,n_groups))
    for i in xrange(n_groups):
        dd_matrix[i,i]=0
        for j in xrange(i+1, n_groups):
            difference = scaled_evecs[i,:] - scaled_evecs[j,:]
            dd_matrix[i,j] = np.sqrt(np.dot(difference,difference))
    return dd_matrix + dd_matrix.T

def plot_diffusion(evals, evecs, t=None, bound=None, title='Diffusion', color=None):
    if not t:
        t = 1 / (1-evals[1])
    x = evecs[:,1]*(evals[1]**t)
    y = evecs[:,2]*(evals[2]**t)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    lib_nums = library_nums(libraries)
    ax.scatter(x, y, color=color)
    ax.set_title(title)
    if bound:
        ax.set_xlim(-bound,bound)
        ax.set_ylim(-bound,bound)
    plt.show()

def plot_diffusion_3d(evals, evecs, t=None, bound=None, title='Diffusion', color=None):
    if not t:
        t = 1 / (1-evals[1])
    x = evecs[:,1]*(evals[1]**t)
    y = evecs[:,2]*(evals[2]**t)
    z = evecs[:,3]*(evals[3]**t)
    fig = plt.figure()
    ax = fig.add_subplot(111,projection="3d")
    ax.scatter3D(x, y, z, color = color)
    ax.set_title(title)
    if bound:
        ax.set_xlim3d(-bound,bound)
        ax.set_ylim3d(-bound,bound)
        ax.set_zlim3d(-bound,bound)
    plt.show()

def scatter_affinities(affinities, lib_matrices, pair_colors, log_scale = False, title='Scatter'):   
    fig = plt.figure(figsize=(10,8))
    xs = np.arange(len(pair_colors))
    plt.scatter(xs, affinities, s=2, color = pair_colors)
    plt.title(title)
    plt.ylim(1e-6,1)
    if log_scale == True:
        plt.yscale('log')
    for i in xs:
        if lib_matrices[i] == 3:
            plt.axvline(x=i,ls='-',alpha = .5,c='g',linewidth=1,)
    plt.show()
    
def plot_graph(matrix, threshold, color=None, complement = False):
    print 'Threshold =', threshold #include this in title
    n_groups = matrix.shape[0]
    labels = dict(zip(range(n_groups),range(n_groups)))
    if complement:
        G = nx.Graph(matrix < threshold)
    else:
        G = nx.Graph(matrix >= threshold)
    pos=nx.spring_layout(G)
    nx.draw_networkx_nodes(G,pos,node_color=color)
    nx.draw_networkx_edges(G,pos)
    nx.draw_networkx_labels(G,pos,labels=labels,font_color='w',font_size=16)
    plt.show()

# read_groups = ['bloop']
# a = ['NexPond-544501'] * 7
# b = ['NexPond-544405'] * 8
# libraries = a + b
# print libraries

def read_metadata(meta_file):
    with open(meta_file, 'r') as f:
        my_json = json.loads(f.read())
    return {json.loads(key): value for (key, value) in my_json.iteritems()}

def read_table(table_file):
    with open(table_file, 'r') as f:
        # This will initially read the keys in the k:v pairs as just strings
        my_json = json.loads(f.read())
    # Convert keys string->list; but cannot hash on a list, so conver to a tuple for use in the map
    return {tuple(json.loads(key)): value for (key, value) in my_json.iteritems()}

def is_greater_than(x, threshold):
    return 1 if x > threshold else 0

meta = read_metadata('test_2_libraries.tabledata.json.read_groups.json')
table = read_table('test2libs.tabledata.json')  

# meta = read_metadata('test_10_libraries.read_groups.json')
# table = read_table('test_10_libraries.table.json')  

groups = []
libraries = []
for key in meta.keys():
    libraries.append(meta[key]['mAttributes']['LB'])
    groups.append(meta[key]['mReadGroupId'])

n_groups = len(groups)

print "floop"

arrays = [np.array(groups), np.array(libraries)]

print "nloop"

ap_affinity = all_pairs_affinity(table, n_groups)

print "bloop"

df = pd.DataFrame(zero_diag(ap_affinity), index=arrays, columns=arrays)

print "gloop"

df = df.sort_index(level=1)
df = df.sort_index(level=1, axis=1)

# print df

# print df.index
# print df.columns

print df.iat[0,0]

is_same = np.vectorize(is_greater_than)
# result = is_same(ap_affinity, 0.5)
result = is_same(df, 0.5)

# print df.values
# print result

truth = np.zeros((n_groups,n_groups))
for i in xrange(n_groups):
    for j in xrange(n_groups):
        if libraries[i] == libraries[j] and i !=j:
            truth[i,j] = 1
            
# print truth

# print truth[truth > 0]

# truthy = pd.DataFrame(truth, index=arrays, columns=arrays)
true_sames = result[truth > 0]
n_same_given_true_same = true_sames.sum() 
p_same_given_true_same = n_same_given_true_same / len(true_sames)
print "p(same-library | truly same-library) = {0:.2f}".format(round(p_same_given_true_same))

true_diffs = result[truth == 0]
n_diff_given_true_diff = len(true_diffs) - true_diffs.sum()
p_diff_given_true_diff = n_diff_given_true_diff / len(true_diffs)
print "p(different-library | truly different-library) = {0:.2f}".format(round(p_diff_given_true_diff))

rand_index = (n_same_given_true_same + n_diff_given_true_diff) / (n_groups * n_groups)
print "Rand index = {0:.2f}".format(round(rand_index,2))

vals_of_sames = df.values[truth>0]
vals_of_diffs = df.values[truth==0]
print vals_of_sames
print vals_of_diffs

my_bins=np.arange(-0.01,1,.01)

plt.hist(vals_of_sames, bins=my_bins, alpha=0.5, label='Same-library')
plt.hist(vals_of_diffs, bins=my_bins, alpha=0.5, label='Different-library')
plt.legend(loc='upper right')
# plt.hist(ap_affinity, bins=np.arange(,1,.001))
plt.show()

# print np.matrix([[result[i,j] for i in xrange(n_groups)] for j in xrange(n_groups)])


# print "floop"

# failed = False
# for i in xrange(n_groups):
#     for j in xrange(i, n_groups):
#         libi = libraries[i]
#         libj = libraries[j]
#         test = result[i][j]
#         truth = 1 if libi==libj else 0
# #         print i,j, test, truth
        
#         if test!=truth: failed = True
            
# print failed
        

# plot_matrix(zero_diag(ap_affinity), title='All pairs affinity')
# plot_matrix(df, title='All pairs affinity')

print "FLOMP"

In [5]:
import numpy as np

thing = np.array([1,0,-1])
np.abs(thing).sum()

2